In [6]:
from agent import clean_html_for_llm_async, extract_pure_json
from geminy_agent import parse_repertoires_from_page
from google import genai
import os
from dotenv import load_dotenv
import re
import json

load_dotenv();

In [ ]:
url = "https://teatrdramatyczny.pl/repertuar"
content = await clean_html_for_llm_async(url)

/Users/markogolovko/Projects/TeatrApka/scraper/agent.py:13: RuntimeWarning: coroutine 'Page.wait_for_timeout' was never awaited
  page.wait_for_timeout(5000)
/Users/markogolovko/Projects/TeatrApka/scraper/agent.py:15: RuntimeWarning: coroutine 'Browser.close' was never awaited
  browser.close()


In [6]:
client = genai.Client(api_key=os.environ["GOOGLE_API_KEY"])

In [ ]:
performances = parse_repertoires_from_page(content, client)

In [20]:
response = """
[{
"title": "PIOTRUŚ PAN",
"date": "2024-12-06"
},
{
"title": "KRUK Z TOWER",
"date": "2024-12-07"
}]
"""
# response = """
# [{
# "title": "PIOTRUŚ PAN",
# "date": "2024-12-06",
# "start_time": "15:00",
# "end_time": null,
# "status": null,
# "place": "Duża Scena PKiN, pl. Defilad 1"
# }]
# """

In [21]:
def extract_pure_json(text: str) -> list[dict]:
    """Extract JSON content from text that might be wrapped in markdown code blocks."""
    json_pattern = r"```(?:json)?\s*([\s\S]*?)```|```(?:json)?\s*([\s\S]*)"
    match = re.search(json_pattern, text)
    if match:
        json_text = (match.group(1) or match.group(2)).strip()
    else:
        json_text = text.strip()
    try:
        return json.loads(json_text)
    except json.JSONDecodeError as e:
        print(f"Initial JSONDecodeError: {e}. Attempting repair...")
    return _truncate_to_valid_array(json_text)


def _truncate_to_valid_array(text: str) -> list[dict]:
    """Truncate a JSON array string to the last valid JSON object and close the array."""
    start = text.find('[')
    if start == -1:
        raise ValueError("No JSON array found.")
    array_body = text[start:]
    repaired_objects = []
    object_texts = re.split(r'},\s*{', array_body[1:-1])
    for obj in object_texts:
        obj = obj.strip()
        if not obj.startswith('{'):
            obj = '{' + obj
        if not obj.endswith('}'):
            obj = obj + '}'
        try:
            parsed = json.loads(obj)
            repaired_objects.append(parsed)
        except json.JSONDecodeError:
            break
    return repaired_objects


extract_pure_json(response)

[{'title': 'PIOTRUŚ PAN', 'date': '2024-12-06'},
 {'title': 'KRUK Z TOWER', 'date': '2024-12-07'}]

In [11]:
response.strip()

'[{\n"title": "PIOTRUŚ PAN",\n"date": "2024-12-06",\n"start_time": "15:00",\n"end_time": null,\n"status": null,\n"place": "Duża Scena PKiN, pl. Defilad 1"\n},\n{\n"title": "KRUK Z TOWER",\n"date": "2024-12-07",\n"start_time": "18:3'